# Load and Preprocess Data
Load the data from a text file and preprocess it to facilitate further analysis. This includes splitting the data into individual articles.

In [196]:
# Load the data from a text file
with open('../b-acumulados_1_p3hr9rqyh4.txt', 'r', encoding='ISO-8859-1') as file:
    data = file.read()

# Preprocess the data
# Split the data into individual articles
articles = data.split('========================================================')

# Filter out articles with invalid content markers
valid_articles = [article for article in articles if 'content-no-valid' not in article and 'no-content-2' not in article]


# Further split each article into metadata and content
processed_articles = []
dates_array = []
for article in valid_articles:
    parts = article.split('\n', 1)
    date = article.split('\n')[0]
    print(date)
    dates_array.append(date)
    if len(parts) > 1:
        metadata, content = parts
        processed_articles.append({'metadata': metadata.strip(), 'content': content.strip()})



[Sábado, 13 de abril de 2024]

[Viernes, 12 de abril de 2024]

[Viernes, 12 de abril de 2024]

[Viernes, 12 de abril de 2024]

[Viernes, 12 de abril de 2024]

[Viernes, 12 de abril de 2024]

[Viernes, 12 de abril de 2024]

[Viernes, 12 de abril de 2024]

[Viernes, 12 de abril de 2024]

[Viernes, 12 de abril de 2024]

[Viernes, 12 de abril de 2024]

[Viernes, 12 de abril de 2024]

[Viernes, 12 de abril de 2024]

[Viernes, 12 de abril de 2024]

[Viernes, 12 de abril de 2024]

[Viernes, 12 de abril de 2024]

[Viernes, 12 de abril de 2024]

[Viernes, 12 de abril de 2024]

[Viernes, 12 de abril de 2024]

[Viernes, 12 de abril de 2024]

[Viernes, 12 de abril de 2024]

[Viernes, 12 de abril de 2024]

[Viernes, 12 de abril de 2024]

[Viernes, 12 de abril de 2024]

[Viernes, 12 de abril de 2024]

[Viernes, 12 de abril de 2024]

[Jueves, 11 de abril de 2024]

[Jueves, 11 de abril de 2024]

[Jueves, 11 de abril de 2024]

[Jueves, 11 de abril de 2024]

[Jueves, 11 de abril de 2024]

[Lunes, 15 de

In [197]:
for valid_articles_for in processed_articles:
    #print(valid_articles_for['metadata'])
    print(valid_articles_for['content'])
    
    print('========================================================')


#Carabaya Nueve meses de prisión preventiva contra sujeto que asesino de madre de sus seis hijos .
El  Juzgado de Investigación Preparatoria de Macusani, dictó 9 meses de prisión preventiva en contra de Juan Vargas Zara (34), por el presunto delito de feminicidio en Carabaya. 
El 28 de febrero del 2024, el imputado acabó con la vida de su esposa y madre de sus menores hijos, Benita Flores Molina (30).
Aquel día, la mujer había preparado un mate con exceso de azúcar, hecho que enfureció al sujeto, quien tomó un tubo de acero para golpearle la cabeza y taparle la boca con cenizas para luego impactar la zona craneal de Benita contra el suelo, el hecho se registró en la comunidad Altiplano Chia, del distrito de Ollachea.
La mujer falleció en el hospital de Macusani, después de 4 días de agonía, dejando en orfandad a 6 menores hijos. Mientras tanto, el imputado paso por cadena ronderil por más de un mes en Carabaya.
{{24/06/2024 23:17:13}}
{Date obtained OK. Try 2}
#Puno Efectivos de seren

In [198]:
print('Total articles:', len(processed_articles))

Total articles: 1164


In [199]:
import re

for article in processed_articles:
    content = article['content']
    
    # Extraer el título
    title_end_index = content.find('\n')  # Encuentra el final del título
    title = content[1:title_end_index]  # Omite el primer carácter '#'
    
    # Extraer la ciudad
    city_end_index = title.find(' ')  # Encuentra el final de la ciudad
    city = title[:city_end_index]  # Extrae la ciudad
    
    # Extraer la fecha
    date_pattern = r"\{\{(\d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2})\}\}"
    date_match = re.search(date_pattern, content)
    date = date_match.group(1) if date_match else "Fecha no encontrada"
    
    # Extraer el resumen (simplificado como el texto hasta la fecha)
    summary_end_index = content.find('{{') if date_match else title_end_index + 1
    summary = content[title_end_index+1:summary_end_index].strip()
    
    print(f'Ciudad: {city}')
    print(f'Título: {title}')
    print(f'Resumen: {summary}')
    print(f'Fecha: {date}')
    print('========================================================')

Ciudad: 
Título: 
Resumen: 
Fecha: Fecha no encontrada
Ciudad: Carabaya
Título: Carabaya Nueve meses de prisión preventiva contra sujeto que asesino de madre de sus seis hijos .
Resumen: El  Juzgado de Investigación Preparatoria de Macusani, dictó 9 meses de prisión preventiva en contra de Juan Vargas Zara (34), por el presunto delito de feminicidio en Carabaya. 
El 28 de febrero del 2024, el imputado acabó con la vida de su esposa y madre de sus menores hijos, Benita Flores Molina (30).
Aquel día, la mujer había preparado un mate con exceso de azúcar, hecho que enfureció al sujeto, quien tomó un tubo de acero para golpearle la cabeza y taparle la boca con cenizas para luego impactar la zona craneal de Benita contra el suelo, el hecho se registró en la comunidad Altiplano Chia, del distrito de Ollachea.
La mujer falleció en el hospital de Macusani, después de 4 días de agonía, dejando en orfandad a 6 menores hijos. Mientras tanto, el imputado paso por cadena ronderil por más de un mes 

In [200]:
# insertar titulo, ciudad, resumen y fecha en la base de datos mysql llamada 'big_data'
import mysql.connector

# Conectar a la base de datos
connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database='big_data'
)

# Crear un cursor
cursor = connection.cursor()

# Crear la tabla si no existe
cursor.execute('''
    CREATE TABLE IF NOT EXISTS articles (
        id INT AUTO_INCREMENT PRIMARY KEY,
        city VARCHAR(255),
        title TEXT,
        summary TEXT,
        date TEXT
    )
''')

# Insertar los datos

for article in processed_articles:
    content = article['content']
    
    # Extraer el título
    title_end_index = content.find('\n')  # Encuentra el final del título
    title = content[1:title_end_index]  # Omite el primer carácter '#'
    
    # Extraer la ciudad
    city_end_index = title.find(' ')  # Encuentra el final de la ciudad
    city = title[:city_end_index]  # Extrae la ciudad
    
    # Extraer el resumen (simplificado como el texto hasta la fecha)
    summary_end_index = content.find('{{') if date_match else title_end_index + 1
    summary = content[title_end_index+1:summary_end_index].strip()
    
    cursor.execute('''
        INSERT INTO articles (city, title, summary, date)
        VALUES (%s, %s, %s, %s)
    ''', (city, title, summary, date))

for date in dates_array:
    cursor.execute('''
        INSERT INTO articles (date)
        VALUES (%s)
    ''', (date,))

# Guardar los cambios
connection.commit()

# Cerrar la conexión
cursor.close()


True

: 